In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers ,models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

from PIL import Image

In [ ]:
df = pd.read_csv('./data.csv')
test = pd.read_csv('./../../test.csv')

In [ ]:
test = test[test['Category']=='Women Tops & Tunics']

In [7]:
target = [f'attr_{i+1}' for i in range(10)]

In [8]:
label_encoders = {}

# Encode the columns
for col in target:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [40]:
# For Kaggle

df['image_path'] = df.apply(lambda x: './../../Data/train_images/' + x['image_path'].split('/')[-1], axis=1)
test['image_path'] = test.apply(lambda x: './../../Data/test_images/' + x['image_path'].split('/')[-1], axis=1)

In [12]:
unique_label_dict = {}

In [13]:
for i in range(10):
    unique_label_dict[f'attr_{i+1}'] = df[f'attr_{i+1}'].nunique()

In [15]:
labelcolumns = []
for i in df.columns:
    if i.startswith('attr'): labelcolumns.append(i)

In [16]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
])

In [17]:
def prepare_images(image_paths, image_size=(224, 224)):
    """
    Converts image paths into augmented tensors using predefined data augmentation layers.
    
    Args:
        image_paths (list of str): List of image paths.
        image_size (tuple): Target size of images (height, width).
    
    Returns:
        np.array: Array of preprocessed and augmented image tensors.
    """

    images = []
    
    for path in image_paths:
        try:
            print('Preparing ' , path)
            img = Image.open(path).convert("RGB")
            img = img.resize(image_size)  
            img_array = np.array(img) / 255.0  
            
            # Add batch dimension and convert to tensor
            img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)
            img_tensor = tf.expand_dims(img_tensor, axis=0)  
            
            # Apply data augmentation
            augmented_img = data_augmentation(img_tensor)[0].numpy()  
            images.append(augmented_img)

        except Exception as e:
            print(f"Error loading image {path}: {e}")

    return np.array(images)


In [33]:
def train_multi_output_model(x_train, y_train, unique_label_dict, epochs=10, batch_size=32):
    """
    Trains a multi-output model for predicting all attributes at once (multi-label classification).
    
    Args:
        x_train (np.array): Training images.
        y_train (np.array): Training labels, 2D array (num_samples, num_outputs).
        unique_label_dict (dict): Dictionary where each value is the number of unique classes per output.
        epochs (int): Number of epochs for training.
        batch_size (int): Batch size for training.
    
    Returns:
        tf.keras.Model: The trained model.
    """
    base_model = ResNet50(weights='./../../resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                          include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    # Shared layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    
    # Outputs for each target column
    outputs = [Dense(num_classes, activation='softmax', name=f'output_{i}')(x) 
               for i, num_classes in enumerate(unique_label_dict.values())]
    
    # Define the model
    model = models.Model(inputs=base_model.input, outputs=outputs)
    
    # Compile the model
    model.compile(
    optimizer='adam', 
    loss=['sparse_categorical_crossentropy'] * len(outputs),  # One loss per output
    metrics=[['accuracy']] * len(outputs)  # One accuracy metric per output
)
    
    # Restructure y_train into a list
    y_train_list = [y_train[:, i] for i in range(y_train.shape[1])]
    
    # Train the model
    print("Training...")
    model.fit(x_train, y_train_list, epochs=epochs, batch_size=batch_size, verbose=1)
    
    return model


In [19]:
image_paths = df['image_path'].values
x_train = prepare_images(image_paths)
y_train = df[target].values

Preparing  ./../../Data/train_images/025778.jpg
Preparing  ./../../Data/train_images/025779.jpg
Preparing  ./../../Data/train_images/025780.jpg
Preparing  ./../../Data/train_images/025781.jpg
Preparing  ./../../Data/train_images/025782.jpg
Preparing  ./../../Data/train_images/025783.jpg
Preparing  ./../../Data/train_images/025784.jpg
Preparing  ./../../Data/train_images/025785.jpg
Preparing  ./../../Data/train_images/025786.jpg
Preparing  ./../../Data/train_images/025787.jpg
Preparing  ./../../Data/train_images/025788.jpg
Preparing  ./../../Data/train_images/025789.jpg
Preparing  ./../../Data/train_images/025790.jpg
Preparing  ./../../Data/train_images/025791.jpg
Preparing  ./../../Data/train_images/025792.jpg
Preparing  ./../../Data/train_images/025793.jpg
Preparing  ./../../Data/train_images/025794.jpg
Preparing  ./../../Data/train_images/025795.jpg
Preparing  ./../../Data/train_images/025796.jpg
Preparing  ./../../Data/train_images/025797.jpg
Preparing  ./../../Data/train_images/025

In [35]:
model = train_multi_output_model(x_train , y_train, unique_label_dict)

Training...


c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


214/214 ━━━━━━━━━━━━━━━━━━━━ 190s 827ms/step - loss: 6.0597 - output_0_accuracy: 0.2068 - output_0_loss: 2.2256 - output_1_accuracy: 0.8092 - output_1_loss: 0.4798 - output_2_accuracy: 0.7837 - output_2_loss: 0.5186 - output_3_accuracy: 0.9307 - output_3_loss: 0.2672 - output_4_accuracy: 0.8514 - output_4_loss: 0.4265 - output_5_accuracy: 0.6170 - output_5_loss: 0.6692 - output_6_accuracy: 0.6031 - output_6_loss: 0.6873 - output_7_accuracy: 0.8220 - output_7_loss: 0.5517 - output_8_accuracy: 0.9264 - output_8_loss: 0.2339 - output_9_accuracy: 0.0000e+00 - output_9_loss: 0.0000e+00


In [42]:
image_paths = test['image_path'].values
x_test = prepare_images(image_paths)  

predictions = model.predict(x_test, batch_size=32, verbose=1)

predicted_classes = [np.argmax(output, axis=-1) for output in predictions]

decoded_predictions = {}
for col, preds in zip(target, predicted_classes):  
    le = label_encoders[col]  
    decoded_predictions[col] = le.inverse_transform(preds)

decoded_predictions_df = pd.DataFrame(decoded_predictions)

print("Decoded Predictions DataFrame:")
print(decoded_predictions_df)


Preparing  ./../../Data/test_images/011155.jpg
Preparing  ./../../Data/test_images/011156.jpg
Preparing  ./../../Data/test_images/011157.jpg
Preparing  ./../../Data/test_images/011158.jpg
Preparing  ./../../Data/test_images/011159.jpg
Preparing  ./../../Data/test_images/011160.jpg
Preparing  ./../../Data/test_images/011161.jpg
Preparing  ./../../Data/test_images/011162.jpg
Preparing  ./../../Data/test_images/011163.jpg
Preparing  ./../../Data/test_images/011164.jpg
Preparing  ./../../Data/test_images/011165.jpg
Preparing  ./../../Data/test_images/011166.jpg
Preparing  ./../../Data/test_images/011167.jpg
Preparing  ./../../Data/test_images/011168.jpg
Preparing  ./../../Data/test_images/011169.jpg
Preparing  ./../../Data/test_images/011170.jpg
Preparing  ./../../Data/test_images/011171.jpg
Preparing  ./../../Data/test_images/011172.jpg
Preparing  ./../../Data/test_images/011173.jpg
Preparing  ./../../Data/test_images/011174.jpg
Preparing  ./../../Data/test_images/011175.jpg
Preparing  ./

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


76/77 ━━━━━━━━━━━━━━━━━━━━ 0s 840ms/step

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


77/77 ━━━━━━━━━━━━━━━━━━━━ 69s 857ms/step
Decoded Predictions DataFrame:
     attr_1  attr_2       attr_3 attr_4 attr_5   attr_6   attr_7  \
0       red  a-line  knee length  daily    net  default  default   
1       red  a-line  knee length  daily    net  default  default   
2       red  a-line  knee length  daily    net  default  default   
3       red  a-line  knee length  daily    net  default  default   
4       red  a-line  knee length  daily    net  default  default   
...     ...     ...          ...    ...    ...      ...      ...   
2455    red  a-line  knee length  daily    net  default  default   
2456    red  a-line  knee length  daily    net  default  default   
2457    red  a-line  knee length  daily    net  default  default   
2458    red  a-line  knee length  daily    net  default  default   
2459    red  a-line  knee length  daily    net  default  default   

                     attr_8   attr_9  attr_10  
0     three-quarter sleeves  regular  no attr  
1     three-qu

In [43]:
decoded_predictions_df.head()

,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,red,a-line,knee length,daily,net,default,default,three-quarter sleeves,regular,no attr
1,red,a-line,knee length,daily,net,default,default,three-quarter sleeves,regular,no attr
2,red,a-line,knee length,daily,net,default,default,three-quarter sleeves,regular,no attr
3,red,a-line,knee length,daily,net,default,default,three-quarter sleeves,regular,no attr
4,red,a-line,knee length,daily,net,default,default,three-quarter sleeves,regular,no attr


In [49]:
for i in range(1, 11):  # Loop through columns attr_1 to attr_10
    test[f'attr_{i}'] = decoded_predictions_df[f'attr_{i}'].values

In [ ]:
test.to_csv('prediction.csv', index=False)